# Natural Language Processing : Part 2
## Feature Generation

oleh: 
<br> Ade Satya Wahana
<br> I Gede Yudi Paramartha

Komputer tidak dapat memproses arti dan struktur yang ada dalam suatu teks karena bentuk masih berupa non-numberic. Untuk itu, perlu memproses arti dan struktur dari teks tersebut agar dapat menjadi bentuk angka atau kumpulan angka. 

Hasil proses mengkonversi teks menjadi angka tersebut akan menjadi feature-feature yang dapat digunakan untuk analisis seperti prediksi sentimen dan lain-lain.

Ada banyak teknik feature generation dari teks, namun akan di bahas secara detail salah satu nya yaitu VSM

### VSM (Vector Space Model)

VSM merepresentasikan tiap dokumen (kalimat) dalam sebuah corpus (kumpulan dokumen) sebagai vektor (list/array) di mana setiap element pada vektor tersebut merupakan kemunculan (occurrence) tiap kata/token di dalam dokumen, dimana nilai kemunculan dari tiap kata tersebut dapat pula diberikan pembobotan.

$$ V_{(c)} = \begin{bmatrix}
 w_{1,1}&w_{2,1}&...&w_{T,1} \\ 
 w_{1,2}&w_{2,2}&...&w_{T,2} \\ 
 \vdots&\vdots&&\vdots \\
 w_{1,D}&w_{2,D}&...&w_{T,D}
\end{bmatrix} $$

- c = corpus
- D = jumlah dokumen/kalimat dalam corpus
- T = jumlah vocabulary/term/token yang ada dalam seluruh corpus
- $w_{1,1}$ = bobot kata pertama pada teks pertama

Pertimbangan menggunakan VSM:
- VSM memiliki asumsi bahwa urutan token di dalam teks tidak terlalu berarti, namun kemunculan tokens lah yang menunjukan arti dan pengaruh token tersebut.
- Asumsi tersebut tidak jadi masalah untuk beberapa jenis analisis teks seperti document classification atau clustering. Kumpulan angka dari tiap kata biasanya cukup untuk membedakan semantic concept dari sebuah dokumen.
- Ada beberapa jenis analisis teks yang memerlukan informasi berupa urutan kata dalam kalimat seperti information extraction, POS Tagging dan lain-lain
- VSM akan menghasilkan matrik yang memiliki dimensi sepanjang jumlah kata dalam corpus. Butuh resources untuk menganalisis matrik tersebut.

Cara pembobotan yang umum dilakukan dalam VSM:
- binary values
- bag of word (frequency count)
- TF-IDF weights

In [ ]:
kalimat0= "Data analisis itu penting"
kalimat1= "Data cleaning itu penting, sama penting dengan data analisis"
kalimat2= "Data science melingkupi data analisis dan data cleaning"

gabung = (kalimat0+" "+kalimat1+" "+kalimat2).lower()
vokab = set(gabung.split())
vokab

In [ ]:
binary = {'analisis': [1,1,1],
          'cleaning':[0,1,1],
          'dan': [0,0,1],
          'data':[1,1,1],
          'dengan':[0,1,0],
          'itu':[1,1,0],
          'melingkupi':[0,0,1],
          'penting':[1,1,1], ## isi ya
          'sama':[1,1,1],
          'science':[1,1,1]
         }
pd.DataFrame(binary)

In [ ]:
count = {'analisis': [1,1,1],
          'cleaning':[0,1,1],
          'dan': [0,0,1],
          'data':[1,2,3],
          'dengan':[0,1,0],
          'itu':[1,1,0],
          'melingkupi':[0,0,1],
          'penting':[1,1,1],# isi ya
          'sama':[1,1,1],
          'science':[1,1,1]
         }
pd.DataFrame(count)

#### Term Frequency (TF)

TF = jumlah kemunculan atas satu kata dalam satu dokumen

Kenapa digunakan untuk merepresentasikan sebuah dokumen?
- kata-kata yang sering muncul (selain stopword) itu memiliki tema (thematic)
- jika sebuah kata sering muncul pada sebuah dokumen bertema A, dokumen lain yang mengandung kata tersebut seharusnya memiliki tema A mirip dengan dokumen sebelumnya

Cara-cara menghitung TF:
- binary: 0/1
- raw frequency: $n_{t,d}$
- weighted frequency: $\frac{n_{t,d}}{N_d}$
- sublinear TF: $1 + log(n_{t,d}), 0$ if $n_{t,d} = 0$

In [ ]:
corpus = [kalimat0, kalimat1, kalimat2]

#import fungsi countvectorizer dari sklearn
from sklearn.feature_extraction.text import CountVectorizer

# buat vectorizernya sebagai object, set binary = True juga ingin apply cara hitung binary
countVec = CountVectorizer(binary=False)

# train vectorizernya dengan corpus (pandas series/list/numpy array)
countVec.fit(corpus)

# apply vectorizernya ke corpus (data yang sama) dan simpan hasilnya sebagai object
hasil = countVec.transform(corpus)

# convert hasil menjadi pandas dataframe
# hasil harus di convert menjadi numpy array karena sebelumnya memiliki type sparse matrix
# ubah kolom dengan feature name yang ada pada vectorizer object
pd.DataFrame(hasil.toarray(), columns = countVec.get_feature_names())

#### Inversed Document Frequency (IDF)

Masalah jika hanya menggunakan TF:
- semua kata/token dianggap sama-sama penting
- beberapa kata kadang tidak memiliki atau hanya sedikit kemampuan untuk membedakan antar dokumen.

Misalnya, pada kumpulan tweet dari tenaga medis untuk kasus sentiment analisis terkait vaksinasi, kata "pasien" atau "penyakit" mungkin sering muncul di banyak tweet tapi hanya sedikit berkontribusi untuk  membedakan mana tweet yang bersentimen negatif atau positif.

Untuk mengurangi masalah ini, perlu adanya mekanisme untuk mengurangi bobot kata/token yang terlalu sering muncul dalam suatu dokumen agar menjadi lebih bermakna bagi penentuan relevansi. Idenya adalah untuk mengurangi bobot TF tiap kata/token dengan seiring dengan banyaknya dokumen yang memiliki kata tersebut. Faktor pengurang tersebut dihitung dengan menggunakan rumus IDF sebagai berikut:

$$idf_t = log \left ( \frac{D}{n_t}\right) $$

- D = jumlah dokumen dalam corpus
- n_t = jumlah dokumen dimana terdapat term t

Cara-cara menghitung IDF:
- IDF = $log \left ( \frac{D}{n_t}\right)$
- smoothed IDF = $log \left ( \frac{D}{n_t+1}\right)$
- Sklearn IDF = $log \left ( \frac{D+1}{n_t+1}\right)+1$
- Skleearn IDF, smooth_idf off: $log \left ( \frac{D}{n_t}\right)+1$

#### TF-IDF
gabungan antara TF dan IDF dari sebuah term/kata

$$ TF-IDF = n_{t,d} \times log \left ( \frac{D}{n_t}\right) $$

semakin banyak kata itu muncul di sebuat dokumen/kalimat, semakin relevan kata tersebut untuk membedakan antar dokumen.
namun, semakin banyak kata itu juga muncul di banyak dokumen lain, semakin tidak relevan kata tersebut 

jika suatu kata di temukan dalam seluruh dokumen makan besar tf-idf nya adalah 0, karena idf nya 0($log\left ( \frac{D}{D}\right )$)



In [ ]:
#import fungsi tfidfvectorizer dari sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# buat vectorizernya sebagai object,
tfidfVec = TfidfVectorizer()

# train vectorizernya dengan corpus (pandas series/list/numpy array)
tfidfVec.fit(corpus)

# apply vectorizernya ke corpus (data yang sama) dan simpan hasilnya sebagai object
hasil2 = tfidfVec.transform(corpus)

# convert hasil menjadi pandas dataframe
# hasil harus di convert menjadi numpy array karena sebelumnya memiliki type sparse matrix
# ubah kolom dengan feature name yang ada pada vectorizer object
pd.DataFrame(hasil2.toarray(), columns = tfidfVec.get_feature_names())

Jika menggunakan function dari sklearn, cara perhitungannya agak berbeda dari penjelasan di atas. Namun, prinsip inti TF-IDF akan tetap sama: **TF-IDF memberikan nilai yang lebih besar untuk kata-kata yang lebih jarang (muncul di sedikit dokumen) dan tinggi ketika nilai IDF dan TF tinggi**

#### Vectorizer lainnya
Selain vectorizer-vectorizer yang sudah dijelaskan, terdapat beberapa vectorizer lain yang mungkin dapat dipelajari di luar pelatihan ini. Contoh-contoh nya antara lain:
- hashing vectorizer (sklearn menyediakan fungsinya)
- word2vec (based on Neural Network, biasanya ada yang menyediakan model nya)

## Tugas

gunakan dataset tweet opini film yang sudah dipreprocesing sebelumnya untuk men-generate feature dari text hasil preprocessing. Di bebaskan memilih teknik vectorizer yang mana (binary, bag of words, tf-idf).

setelah featurenya digenerate, buat model supervised learning untuk memprediksi sentiment (positif/negatif). Model yang dipergunakan juga dibebaskan, boleh tree based, linear model, KNN, SVM dll. Tapi, tolong gunakan model yang sesuai dengan case ini (*supervised classification case*)